# NCEI Storm Events Dataset Analysis

Loading and exploring NCEI storm events data from 2024-2025

In [1]:
import pandas as pd
import os
import numpy as np
from pathlib import Path

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

## Load Storm Events Data

In [2]:
# Define paths to the storm events CSV files
data_dir = Path('NCEI_datasets/storm_events')
files = [
    data_dir / 'StormEvents_details-ftp_v1.0_d2024_c20251204.csv',
    data_dir / 'StormEvents_details-ftp_v1.0_d2025_c20251216.csv'
]

# Load both datasets
print("Loading datasets...")
df_list = []
for file in files:
    print(f"  Loading {file.name}...")
    df = pd.read_csv(file)
    print(f"    Shape: {df.shape}")
    df_list.append(df)

# Concatenate the datasets
df_storm = pd.concat(df_list, ignore_index=True)
print(f"\nCombined dataset shape: {df_storm.shape}")

Loading datasets...
  Loading StormEvents_details-ftp_v1.0_d2024_c20251204.csv...
    Shape: (69679, 51)
  Loading StormEvents_details-ftp_v1.0_d2025_c20251216.csv...
    Shape: (62834, 51)

Combined dataset shape: (132513, 51)


## Basic Dataset Overview

In [3]:
print("Column names and types:")
print(df_storm.dtypes)
print("\n" + "="*80)
print("\nFirst few rows:")
print(df_storm.head())

Column names and types:
BEGIN_YEARMONTH         int64
BEGIN_DAY               int64
BEGIN_TIME              int64
END_YEARMONTH           int64
END_DAY                 int64
END_TIME                int64
EPISODE_ID              int64
EVENT_ID                int64
STATE                  object
STATE_FIPS              int64
YEAR                    int64
MONTH_NAME             object
EVENT_TYPE             object
CZ_TYPE                object
CZ_FIPS                 int64
CZ_NAME                object
WFO                    object
BEGIN_DATE_TIME        object
CZ_TIMEZONE            object
END_DATE_TIME          object
INJURIES_DIRECT         int64
INJURIES_INDIRECT       int64
DEATHS_DIRECT           int64
DEATHS_INDIRECT         int64
DAMAGE_PROPERTY        object
DAMAGE_CROPS           object
SOURCE                 object
MAGNITUDE             float64
MAGNITUDE_TYPE         object
FLOOD_CAUSE            object
CATEGORY              float64
TOR_F_SCALE            object
TOR_LENGTH      

In [4]:
print("\nDataset Info:")
print(df_storm.info())
print("\n" + "="*80)
print("\nBasic Statistics:")
print(df_storm.describe())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132513 entries, 0 to 132512
Data columns (total 51 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   BEGIN_YEARMONTH     132513 non-null  int64  
 1   BEGIN_DAY           132513 non-null  int64  
 2   BEGIN_TIME          132513 non-null  int64  
 3   END_YEARMONTH       132513 non-null  int64  
 4   END_DAY             132513 non-null  int64  
 5   END_TIME            132513 non-null  int64  
 6   EPISODE_ID          132513 non-null  int64  
 7   EVENT_ID            132513 non-null  int64  
 8   STATE               132513 non-null  object 
 9   STATE_FIPS          132513 non-null  int64  
 10  YEAR                132513 non-null  int64  
 11  MONTH_NAME          132513 non-null  object 
 12  EVENT_TYPE          132513 non-null  object 
 13  CZ_TYPE             132513 non-null  object 
 14  CZ_FIPS             132513 non-null  int64  
 15  CZ_NAME            

## Missing Values Analysis

In [5]:
missing_data = df_storm.isnull().sum()
missing_percent = (missing_data / len(df_storm)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_data,
    'Percentage': missing_percent
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

print("Missing Values Summary:")
print(missing_df)
print(f"\nTotal missing values: {missing_data.sum()}")

Missing Values Summary:
                    Missing_Count  Percentage
CATEGORY                   132437   99.942647
TOR_OTHER_CZ_FIPS          131893   99.532121
TOR_OTHER_CZ_STATE         131893   99.532121
TOR_OTHER_WFO              131893   99.532121
TOR_OTHER_CZ_NAME          131893   99.532121
TOR_F_SCALE                128867   97.248572
TOR_LENGTH                 128867   97.248572
TOR_WIDTH                  128867   97.248572
FLOOD_CAUSE                117987   89.038057
MAGNITUDE_TYPE              78165   58.986665
MAGNITUDE                   60275   45.486103
END_AZIMUTH                 48584   36.663573
END_RANGE                   48584   36.663573
BEGIN_RANGE                 48584   36.663573
BEGIN_AZIMUTH               48584   36.663573
BEGIN_LOCATION              48584   36.663573
BEGIN_LON                   48584   36.663573
END_LAT                     48584   36.663573
END_LOCATION                48584   36.663573
BEGIN_LAT                   48584   36.663573
END_LON   

## Categorical Data Exploration

In [6]:
# Get categorical columns
categorical_cols = df_storm.select_dtypes(include=['object']).columns

print(f"Categorical columns ({len(categorical_cols)}):")
for col in categorical_cols:
    print(f"\n{col} - Unique values: {df_storm[col].nunique()}")
    print(df_storm[col].value_counts().head(10))

Categorical columns (25):

STATE - Unique values: 68
STATE
TEXAS           10926
OKLAHOMA         6292
MISSOURI         4897
ILLINOIS         4691
KANSAS           4501
VIRGINIA         4239
PENNSYLVANIA     4021
SOUTH DAKOTA     3963
NEW YORK         3947
IOWA             3929
Name: count, dtype: int64

MONTH_NAME - Unique values: 12
MONTH_NAME
May          20509
June         18559
July         18412
January      14018
April        12959
August       12757
March        11614
February     10200
September     5956
December      3039
Name: count, dtype: int64

EVENT_TYPE - Unique values: 52
EVENT_TYPE
Thunderstorm Wind           40897
Hail                        17844
Flash Flood                  9757
High Wind                    7148
Winter Weather               6375
Heat                         5919
Drought                      5572
Flood                        4450
Winter Storm                 4223
Marine Thunderstorm Wind     4129
Name: count, dtype: int64

CZ_TYPE - Unique values: 2

## Numerical Data Exploration

In [7]:
# Get numerical columns
numerical_cols = df_storm.select_dtypes(include=['int64', 'float64']).columns

print(f"Numerical columns ({len(numerical_cols)}):")
print("\nDetailed Statistics:")
print(df_storm[numerical_cols].describe().T)

Numerical columns (26):

Detailed Statistics:
                      count          mean           std           min  \
BEGIN_YEARMONTH    132513.0  2.024527e+05     49.532032  2.024010e+05   
BEGIN_DAY          132513.0  1.515320e+01      8.835707  1.000000e+00   
BEGIN_TIME         132513.0  1.290528e+03    649.935861  0.000000e+00   
END_YEARMONTH      132513.0  2.024527e+05     49.532032  2.024010e+05   
END_DAY            132513.0  1.646557e+01      8.803548  1.000000e+00   
END_TIME           132513.0  1.469714e+03    605.585746  0.000000e+00   
EPISODE_ID         132513.0  1.971696e+05   5791.728807  1.868110e+05   
EVENT_ID           132513.0  1.222332e+06  41922.381389  1.146596e+06   
STATE_FIPS         132513.0  3.366027e+01     19.412851  1.000000e+00   
YEAR               132513.0  2.024474e+03      0.499334  2.024000e+03   
CZ_FIPS            132513.0  1.115970e+02    134.689953  1.000000e+00   
INJURIES_DIRECT    132513.0  1.807370e-02      0.731487  0.000000e+00   
INJUR

## Duplicate Records Check

In [8]:
duplicate_rows = df_storm.duplicated().sum()
print(f"Duplicate rows: {duplicate_rows}")

if duplicate_rows > 0:
    print(f"Percentage of duplicates: {(duplicate_rows / len(df_storm)) * 100:.2f}%")
    print("\nFirst few duplicate records:")
    print(df_storm[df_storm.duplicated(keep=False)].head(10))

Duplicate rows: 0


## Data Summary

In [9]:
print("=" * 80)
print("COMBINED STORM EVENTS DATASET SUMMARY")
print("=" * 80)
print(f"\nTotal Records: {len(df_storm):,}")
print(f"Total Columns: {df_storm.shape[1]}")
print(f"\nMemory Usage: {df_storm.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nColumns: {list(df_storm.columns)}")
print("\nData Quality:")
print(f"  - Duplicate rows: {duplicate_rows}")
print(f"  - Total missing values: {missing_data.sum():,}")
print(f"  - Columns with missing data: {(missing_data > 0).sum()}")
print("=" * 80)

COMBINED STORM EVENTS DATASET SUMMARY

Total Records: 132,513
Total Columns: 51

Memory Usage: 275.25 MB

Columns: ['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH', 'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS', 'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME', 'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE', 'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE', 'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE', 'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE', 'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH', 'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON', 'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'DATA_SOURCE']

Data Quality:
  - Duplicate rows: 0
  - Total missing values: 1,868,856
  - Columns with missing data: 24
